In [1]:
pip install langchain langchain-community langchain-openai langgraph

Defaulting to user installation because normal site-packages is not writeable
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 973.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.0/999.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.5/797.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: pydantic-settings
    Found existing installation: pydantic-settings 2.6.1
    Uninstalling pydantic-settings-2.6.1:
      Successfully uninstalled pydantic-settings-2

In [2]:
pip install langchain-ollama

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# For better security, load environment variables from a .env file
# from dotenv import load_dotenv
# load_dotenv()
# Make sure your OPENAI_API_KEY is set in the .env file

# Initialize the Language Model (using ChatOpenAI is recommended)
llm = ChatOpenAI(temperature=0)

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
   "Extract the technical specifications from the following text:\n\n{text_input}"
)

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
   "Transform the following specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)

# --- Build the Chain using LCEL ---
# The StrOutputParser() converts the LLM's message output to a simple string.
extraction_chain = prompt_extract | llm | StrOutputParser()

# The full chain passes the output of the extraction chain into the 'specifications'
# variable for the transformation prompt.
full_chain = (
   {"specifications": extraction_chain}
   | prompt_transform
   | llm
   | StrOutputParser()
)

# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."

# Execute the chain with the input text dictionary.
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)


In [4]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# No API key needed for Ollama running locally (defaults to http://localhost:11434)
# If your Ollama server is remote, set OLLAMA_BASE_URL, e.g.:
# os.environ["OLLAMA_BASE_URL"] = "http://your-server:11434"

# Initialize the local LLM via Ollama
llm = ChatOllama(model="deepseek-r1:8b", temperature=0)

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
   "Extract the technical specifications from the following text:\n\n{text_input}"
)

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
   "Transform the following specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)

# --- Build the Chain using LCEL ---
extraction_chain = prompt_extract | llm | StrOutputParser()

full_chain = (
   {"specifications": extraction_chain}
   | prompt_transform
   | llm
   | StrOutputParser()
)

# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)



--- Final JSON Output ---



In [5]:
final_result = full_chain.invoke({"text_input": input_text})


In [8]:
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."
prompt_extract = ChatPromptTemplate.from_template(
   "Extract the technical specifications from the following text:\n\n{text_input}"
)

In [11]:
extraction_chain = prompt_extract | llm | StrOutputParser()
extraction_chain.invoke({"text_input": input_text})

'<think>\nOkay, user wants me to extract technical specs from a simple sentence about a laptop. The text is pretty straightforward - just three components mentioned clearly.\n\nHmm, let\'s break this down carefully. First component is the processor: "3.5 GHz octa-core" that\'s two specifications rolled into one - clock speed and core count. Second is RAM: 16GB of it. Third is storage: 1TB NVMe SSD with both capacity and interface type specified.\n\nThe user seems to be looking for clean, structured data extraction rather than analysis or interpretation. They probably just need the specs listed neatly without extra fluff. Maybe they\'re comparing products or writing a review?\n\nI should present this in a way that\'s easy to copy-paste while keeping all technical details intact. The processor part might confuse some users - I wonder if they\'d prefer it split into separate clock speed and core count items? But the original phrasing combines them naturally, so better keep it as is.\n\nLe

In [12]:
# pip install langchain-ollama langchain-core

import re
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# --- LLMs (same model, different configs) ---
# Plain responses (we'll still strip <think> just in case)

llm = ChatOllama(model="deepseek-r1:8b", temperature=0)
# Enforce strict JSON for the second step
llm_json = ChatOllama(model="deepseek-r1:8b", temperature=0, format="json")

# --- Helpers ---
def strip_think(text: str) -> str:
    # Removes any <think>...</think> blocks DeepSeek might include
    return re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL | re.IGNORECASE)

stripper = RunnableLambda(strip_think)

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_messages([
    ("system", "You are a precise information extractor. Do NOT reveal your reasoning or chain-of-thought."),
    ("human", "Extract the technical specifications from the following text:\n\n{text_input}")
])

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_messages([
    ("system", "You convert data to STRICT JSON. Do NOT reveal reasoning. Output JSON only, no extra text."),
    ("human", "Transform the following specifications into a JSON object with keys exactly 'cpu', 'memory', and 'storage':\n\n{specifications}")
])

# --- Build Chains ---
extraction_chain = prompt_extract | llm | StrOutputParser() | stripper

full_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm_json                 # format="json" enforces JSON-only output
    | StrOutputParser()
    | stripper
)

# --- Run ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)



--- Final JSON Output ---
{


"cpu": "Octa-core, 3.5 GHz",
"memory": "16GB",
"storage": "1TB NVMe SSD"
}
